In [1]:
import os
spark_version = 'spark-3.1.2'
os.environ['SPARK_VERSION']=spark_version
os.environ["JAVA_HOME"] = "C:\Program Files\Java\jdk-11.0.11"
os.environ["SPARK_HOME"] = f"C:\spark\spark-3.1.2-bin-hadoop3.2"
import findspark
import pyspark
from pyspark.sql import SparkSession
findspark.init()
spark = SparkSession.builder.appName("emissionsdataframe").getOrCreate()
from sqlalchemy import create_engine, insert
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import MetaData, update, Table
from sqlalchemy.orm import Session
from sqlalchemy.orm import sessionmaker
from sqlalchemy.exc import SQLAlchemyError
import pandas as pd
import config
from pyspark.sql.types import IntegerType,BooleanType,DateType
from pyspark.sql.functions import col
from pyspark.sql import Row


In [2]:
rds_string = config.rds_string
engine = create_engine(f'postgresql://{rds_string}')
conn = engine.connect()
metadata = MetaData(engine)

In [3]:
def sql_to_spark(table):
    from pyspark.sql import Row
    from sqlalchemy.orm import Session, sessionmaker
    from sqlalchemy import MetaData, Table
    metadata = MetaData(engine)
    table_var = Table(table, metadata, autoload=True, autoload_with=engine)
    Session = sessionmaker()
    Session.configure(bind=engine)
    session = Session()
    query = session.query(table_var).all()
    steve = table_var.metadata.tables[table].columns.keys()
    query_list = []
    for i in query:
        q_len = len(i)
        temp_dict = {}
        for j in range(q_len):
            key = steve[j]
            value = i[j]
            if value == None:
                value = float(0)
            temp_dict[key] = value
            if j == (q_len - 1):
                query_list.append(temp_dict)
    df = spark.createDataFrame(Row(**x) for x in query_list)
    return df

def query_maker(table):
    from sqlalchemy.orm import Session, sessionmaker
    from sqlalchemy import MetaData, Table
    metadata = MetaData(engine)
    table_var = Table(table, metadata, autoload=True, autoload_with=engine)
    Session = sessionmaker()
    Session.configure(bind=engine)
    session = Session()
    query = session.query(table_var).all()
    steve = table_var.metadata.tables[table].columns.keys()
    return (query, steve)

def spark_maker(query,country):
    c_list = []
    for i in query:
        q_len = len(i)
        if i[0] == country:
            temp_dict = {}
            for j in range(q_len):
                key = steve[j]
                value = i[j]
                if value == None:
                    value = float(0)
                temp_dict[key] = value
                if j == (q_len - 1):
                    c_list.append(temp_dict)
    df = spark.createDataFrame(Row(**x) for x in c_list) 
    return df

In [4]:
global_trade = Table("global_trade", metadata, autoload=True, autoload_with=engine)
trade_schema = global_trade.metadata.tables["global_trade"].columns.keys()


In [8]:
from sqlalchemy.orm import Session, sessionmaker
from sqlalchemy import MetaData, Table
metadata = MetaData(engine)
Session = sessionmaker()
Session.configure(bind=engine)
session = Session()

In [13]:
def sql_spark_trade(table, country):
    from sqlalchemy.orm import Session, sessionmaker
    from sqlalchemy import MetaData, Table
    metadata = MetaData(engine)
    table_var = Table(table, metadata, autoload=True, autoload_with=engine)
    Session = sessionmaker()
    Session.configure(bind=engine)
    session = Session()
    query = session.query(table_var).filter(table_var.c.country_or_area == country).all()
    steve = table_var.metadata.tables[table].columns.keys()
    c_list = []
    for i in query:
        q_len = len(i)
        if i[0] == country:
            temp_dict = {}
            for j in range(q_len):
                key = steve[j]
                value = i[j]
                if value == None:
                    value = float(0)
                temp_dict[key] = value
                if j == (q_len - 1):
                    c_list.append(temp_dict)
    df = spark.createDataFrame(Row(**x) for x in c_list)
    return df

In [14]:
india_trade_spark = sql_spark_trade("global_trade","India")

In [15]:
india_trade_spark.show()

+---------------+----+---------+--------------------+------+---------+-----------+-------------------+-----------+--------------------+
|country_or_area|year|comm_code|           commodity|  flow|trade_usd|  weight_kg|      quantity_name|   quantity|            category|
+---------------+----+---------+--------------------+------+---------+-----------+-------------------+-----------+--------------------+
|          India|2016|   210110|Coffee extracts, ...|Import| 13371521|  1881816.0|Weight in kilograms|  1881816.0|21_miscellaneous_...|
|          India|2016|   210110|Coffee extracts, ...|Export|276962932|4.0752197E7|Weight in kilograms|4.0752197E7|21_miscellaneous_...|
|          India|2016|   210120|Tea and mate extr...|Import|  1988772|   105363.0|Weight in kilograms|   105363.0|21_miscellaneous_...|
|          India|2016|   210120|Tea and mate extr...|Export| 43467004|  6351410.0|Weight in kilograms|  6351410.0|21_miscellaneous_...|
|          India|2016|   210130|Chicory & other 

In [9]:

query = session.query(global_trade).filter(global_trade.c.country_or_area == "France").all()

In [10]:
query

[('France', 2016, '210110', 'Coffee extracts, essences, concentrates, preparations', 'Import', 199658805, 22502024.0, 'Weight in kilograms', 22502024.0, '21_miscellaneous_edible_preparations'),
 ('France', 2016, '210110', 'Coffee extracts, essences, concentrates, preparations', 'Export', 180938273, 18539751.0, 'Weight in kilograms', 18539751.0, '21_miscellaneous_edible_preparations'),
 ('France', 2016, '210120', 'Tea and mate extracts, essences and concentrates', 'Import', 60589925, 4704236.0, 'Weight in kilograms', 4704236.0, '21_miscellaneous_edible_preparations'),
 ('France', 2016, '210120', 'Tea and mate extracts, essences and concentrates', 'Export', 5963475, 939019.0, 'Weight in kilograms', 939019.0, '21_miscellaneous_edible_preparations'),
 ('France', 2016, '210130', 'Chicory & other coffee substitutes, roasted & product', 'Import', 12158677, 1722793.0, 'Weight in kilograms', 1722793.0, '21_miscellaneous_edible_preparations'),
 ('France', 2016, '210210', 'Yeasts, active', 'Impor

In [ ]:
sess.query(User).filter(User.age == 25)

In [ ]:
# metadata = MetaData(engine)

# global_trade = Table("global_trade", metadata, autoload=True, autoload_with=engine)
# Session = sessionmaker()
# Session.configure(bind=engine)
# session = Session()
# query = session.query(global_trade).all()

In [ ]:
sparkDF.filter(sparkDF["Country "] == "Afghanistan").select(['Country ']).show()

In [ ]:
sparkDF.select(['Country ']).show()

In [ ]:
spark_gdp=sql_to_spark('gdp_data')

In [ ]:
spark_india=sql_to_spark("india_export_data")

In [ ]:
spark_emissions=sql_to_spark('global_emissions')

In [ ]:
germany_spark = spark_maker(query, "Germany")

In [ ]:
new_germany_spark = germany_spark.withColumn("year",germany_spark.year.cast('int'))

In [ ]:
new_germany_spark.printSchema()

In [ ]:
df.orderBy(desc("age"), "name").collect()

In [ ]:
new_germany_spark.orderBy(new_germany_spark.year.asc()).show(1)